In [ ]:
# Transformer 모델 구축 - Transformer News Summary 요약 모델
# 학습 목표 - 실무에서 사용되는 파이프라인 이해 및 적용
# - 1. 데이터 로드 & 확인: 결측치 제거(None, "")

In [4]:
# 데이터셋 로드 및 json 파일 추출
import os
import json

data_dir = './llm_data/ai_hub_news_summary' # 데이터셋 경로

all_data = []

for file_name in os.listdir(data_dir): # 파일명 추출
    if file_name.endswith('.json'): # .json 파일 추출
        with open(os.path.join(data_dir, file_name), 'r', encoding='utf-8') as f: # 파일 일기
            data = json.load(f) # json 파일 로드
            all_data.append(data)

In [21]:
# JSON 파싱: 본문과 요약 추출
dataset = [] # 최종적으로 본문과 요약문을 담는 리스트

for item in all_data: # JSON 데이터셋    
    passage = item['Meta(Refine)']['passage'] # 본문

    summaries = [] # 요약문
    if item['Annotation'].get('summary1'):
        summaries.append(item['Annotation'].get('summary1'))
    if item['Annotation'].get('summary2'):        
        summaries.append(item['Annotation'].get('summary2'))

    # 요약이 없는 경우 제외
    if passage and summaries:
        dataset.append({'text': passage, 'summary': summaries})

In [24]:
# 데이터 전처리
def clean_text(text):
    # 줄바꿈 -> 공백으로 바꾼다, 문장열 앞뒤 불필요한 공백 제거
    text = text.replace('\n', ' ').strip()
    return text

for sample in dataset:
    sample['text'] = clean_text(sample['text']) # 문자열
    sample['summary'] = [ clean_text(s) for s in sample['summary'] ] # 리스트

# 데이터 확인
print(len(dataset), dataset[0])

10800 {'text': '보수진영 사분오열 속 ‘국민통합연대’ 띄운 비박계 크리스마스를 앞둔 지난 23일 오전 서울 프레스센터 국제회의실.   보수분열 극복을 내건 ‘국민통합연대’가 창립대회를 열었다.   햇살 없이 착 가라앉은 날씨에 동지 바람이 매서웠지만 행사장 안은 달아올랐다.   문재인 정권을 향한 맹폭격이 이어졌고 ‘무능, 기만의 오만방자한 정권에 사망을 선고한다’는 창립선언문이 나왔다.   홍준표 전 자유한국당 대표 등 전현직 의원 20여명을 포함해 500여명이 자리를 빼곡하게 메웠다.   총선이 불과 석달 남짓이다.   야권 인사들이 정권을 두들겨 패는 거야 이상한 일이 아니다.   눈 여겨 볼 대목은 모인 사람이 대부분 친이·비박계(친이명박·비박근혜) 인사들이란 점이다.   박관용 전 국회의장, 이문열 작가와 함께 보수쪽 명망가 여럿이 이름을 올리고 더러 참석했다.   전광훈 목사는 축사를 했다.   그래도 이명박 정권서 요직을 맡았던 사람들이 주축이다.   이재오 중앙집행위원장과 홍준표 전 대표가 한가운데 있다.   두 사람은 ‘친박 그룹’에 둘러싸인 황교안 대표와 한국당에 불편한 기색을 감추지 않는 중이다.   홍 전 대표는 이튿날 “무기력한 야당만 믿고 따르기엔 너무 답답하고 앞날이 보이지 않아 창립한 게 국민통합연대”란 글을 올렸다.   31일엔 “한국당 지도부는 총사퇴하고 비상대책위를 꾸려야 한다”고 황 대표 사퇴를 요구했다.   이재오 위원장은 지난 10월3일 광화문의 조국 규탄집회장에서 “자유한국당은 집회에서 빠지라”고 외쳤다.   가뜩이나 뿔뿔이 흩어진 각자도생의 보수세력이다.   한국당과 우리공화당에다 새로운 보수당, 이언주 신당, 이정현 신당이 나올 판이다.   게다가 개정선거법의 준연동형 비례대표제는 군소정당에 유리한 분열요인이다.   중앙선관위에 등록된 정당이 34개인데 창당준비위원회를 설립한 예비정당만 16개에 달한다.   야권 빅텐트를 외칠만한 상황이긴 하다.   그런데 통합을 내건 이재오 위원장은 “어느 한 정당이나

In [ ]:
# 토크나이저 적용
from transformers import BartTokenizer

# BART 토크라이저 로드, Bart vocab size: 50265
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
# print('Bart vocab size:', tokenizer.vocab_size)

# 입력과 요약을 토큰화
inputs = tokenizer(
    [ sample['text'] for sample in dataset ], # 본문 리스트
    max_length=1024, # BART 입력 최대 길이
    truncation=True,
    padding='max_length',
    return_tensors='pt'
)

labels = tokenizer(
    [ ' '.join(sample['summary']) for sample in dataset ], # 요약 리스트를 하나의 문자열로 합치기
    max_length=128, # 요약 최대 길이
    truncation=True,
    padding='max_length',
    return_tensors='pt'
)

print('입력 텐서 크기:', inputs['input_ids'].shape)
print('라벨 텐서 크기:', labels['input_ids'].shape)

입력 텐서 크기: torch.Size([10800, 1024])
라벨 텐서 크기: torch.Size([10800, 128])
